# CH 3
- 클린 코드는, 세부 구현의 모범 사례가 아님. 코드가 디자인이고 디자인이 코드다(?)
- 궁극적인 클린코드의 두 가지 목표: 결함 최소화, 완전히 자명하도록 하기.
- 훌륭한 소프트 웨어 디자인을 위한 원칙에 대해 다루는 챕터. 하지만 "항상 모든 것을 적용해야하는 것은 아님"
- 계약에 의한 설계?와 대비되는 방어적 프로그램이 그 예시.


### 이장의 목표 
  1. 견고한 소프트웨어의 개념 이해
- 작업 중의 잘못된 데이터 다루는 법
- 새로운 요구사항을 확장하기 쉬운 소프트웨어 설계
- 재사용 가능한 소프트웨어 설계
- 개발팀의 생산성 높이는 "효율적"코드 작성

### 계약에 의한 디자인
- 관계자가 기대하는 바를 암묵적으로 코드에 삽입하는 대신 양측이 동의하는 계약을 먼저 한 다음, 계약을 어겼을 경우는 명시적으로 왜 계속할 수 없는지 예외를 발생시키라는 것.
- 예를 들어 API에 이상한 파라미터 넣었을 때 함수 호출에 실패해야함.
- 계약 조건은 주로 사후조건 & 사전 조건을 명시, 때로는 불변식 & 부작용 기술하기도 함
- 가장 중요한 것은 "잘못된 가정 하에 코드의 핵심이 실행되는 것을 방지"하는 데에 있음.
- 보통 사전 조건 검증에 실패하면 클라이언트의 결함, 사후조건 검증에 실패하면 특정 모듈/클래스 자체의 문제일 확률이 높음

  1. 사전조건
    - 함수가 진행 되기 전에 처리되는 모든 조건 체크. 일반적으로 데이터 유효성 검사.
    - 부작용 최소화를 위해 유효성 검사를 많이 하는 것이 좋음
    - 사전 조건은 데이터의 유효성 검사를 어디서 할지가 문제이다. 
    - 클라이언트가 함수를 호출하기 전에 유효성 검사를 하도록 할 것인지, 함수가 자체적으로 로직을 실행하기 전에 검사하도록 할 것인지에 대한 문제이다. 
    - 전자는 관용적인 접근법, 후자는 까다로운 접근법에 해당한다. 전자의 경우 깨진 데이터라도 일단 수용하기 때문에 후자의 방법이 일반적으로 가장 안전하고 견고하다.
  2. 사후 조건
    - 함수 반환 값의 유효성 검사
  3. 불변식
    - 함수가 실행되는 동안 일정하게 유지되는 것.
    - 함수 로직의 문제 확인
    - 함수의 docstring에 이를 기술해놓는 것이 좋음.
    - 예. while문의 isRunning과 같은 함수를 작성하여 잘 작동하는지 체크
  4. 부작용
    - 선택적으로 코드의 부작용을 언급하기도 함
    
#### 파이썬스러운 계약
- 사전 사후 조건 검사와 핵심 기능을 분리하는 것이 좋음
- 더 작은 함수로 생성할 수도 있지만, 데코레이터를 사용하여 분리하는 것도 대안임

In [31]:
def func(a: int, b: str)->str: #관용적인 접근
    '''
    a를 string으로 바꾼뒤 b와 결합하는 함수
    '''
    if not isinstance(a, int) or not isinstance(b, str): #까다로운 접근
        raise ValueError("입력 파라미터 타입이 올바르지 않습니다.")
        
#단, 중복 제거 원칙을 잊지 말아야함. 사전 조건 검증을 클라이언트에게 맡기거나, 함수 자체에 두거나 둘 중 하나만 해야함.
# 이는 뒤의 DRY원칙과 관련이 깊다.

데코레이터로 유효성 검사를 짜봤는데,, 이게 맞나요

In [74]:
def trace(func):                             
    def wrapper(a1: int, b1: str):
        if not isinstance(a1, int) or not isinstance(b1, str): 
            raise ValueError("입력 파라미터 타입이 올바르지 않습니다.")
            
        c=func(a1, b1)   
        
        if not isinstance(c, str) and  not isinstance(c, int): 
            raise ValueError("반환 값 타입이 올바르지 않습니다.")
        return c
    return wrapper
#https://popawaw.tistory.com/134
#파라미터가 있는 함수에 Decorator 적용하기
#=>중첩 함수에 꾸미고자 하는 함수와 동일하게 파라미터를 가져가면 됨
@trace
def func(a1: int, b1: str)->str:
    c = str(a1)+b1
    return c
@trace    
def func2(a1: int, b1:str)->int:
    c= int(a1)*int(b1)
    return c


In [75]:
func2(3,'3')

9

### 방어적 프로그래밍
방어적 프로그래밍은 예외를 발생시키고 실패하게 되는 모든 조건을 기술하는 계약에 의한 디자인과는 달리, 객체, 함수 또는 메서드와 같은 코드의 모든 부분을 유효하지 않은 것으로부터 스스로 보호할 수 있게 하는 것이다.방어적 프로그래밍은 크게 두 가지 주제로 살펴볼 수 있다.

1.예상할 수 있는 시나리오의 오류를 처리하는 방법(에러 핸들링)

2. 발생하지 않아야 하는 오류를 처리하는 방법(assertion)


### 에러핸들링
- 일반적으로 데이터 입력 오류 확인시 사용
- 주요 목적은 "예상되는 에러에 대해서 실행을 계속할 수 있을지 아니면 극복할 수 없어서 프로그램을 중단할 지 결정하는 것"

#### 값 대체

- 잘못된 값을 생성하거나 프로그램 전체가 종료될 위험이 있을 경우 결과 값을 안전한 값으로 대체할 수 있다. 일반적으로 '기본 값(default)'을 사용한다.
- 값 대체 할지 여부는 견고성과 정확성 간의 트레이드 오프임.


In [83]:
#get(x, '디폴트 값') : 가져오려는 값이 없을 경우 디폴드 값 가져오게 하는 함수
configuration = {"db_port": 5432}
print(configuration.get("dbhost", "localhost"))
print(configuration.get("dbport"))
print(configuration.get("dbhost"))
configuration

localhost
None
None


{'db_port': 5432}

In [90]:
#그냥 값 대체 예시인듯
import os
print(os.getenv("DBHOST"))
print(os.getenv("DPORT",5432))


None
5432


- 일반적인 경우 값 대체를 통해 문제 해결이 가능하지만, 민감하고 중요한 정보를 다루는 경우 정확하지 않은 값은 프로그램을 폭파시킬수 있으므로 유의해야 한다. 또한 일반적으로 누락된 파라미터를 기본 값으로 바꾸어도 큰 문제가 없지만 오류가 있는 데이터를 유사한 값으로 대체하는 것은 일부 오류를 숨겨버릴 위험이 있다.(오류가 나야하는데 값 대체로 인해 오류가 발생하지 않는 것이다!)

#### 예외 처리
에러 복구가 아닌 실행 자체를 멈추는 것이 좋은 경우. 
다음은 파이썬의 예외와 관련된 몇 가지 권장 사항임.
- 올바른 수준의 추상화 단계에서 예외처리
- Traceback 노출 금지
- 비어 있는 except 블록 지양
- 원본 예외 포함

##### a. 올바른 수준의 추상화 단계에서 예외 처리
 ConnectionError와 ValueError는 별로 관계가 없는 에러이다. ConnectinError는 connect 메서드에서 처리되어야 하며, ValueError는 event의 decode 메서드에 속한 에러이다. 이렇게 각자의 책임에 맞게 예외 처리를 한다면 deliver_event에서는 예외를 발생시킬 필요가 없다.

In [92]:
def deliver_event(self, event):
	try:
		data = event.decode()
		self.send(data)
	except ConnectionError as e:
		logger.info("연결 실패: %s", e)
		raise
	except ValueError as e:
		logger.error("%r 잘못된 데이터 포함: %s", event, e)
		raise

##### b. Trackback 노출 금지

파이썬에서 traceback은 매우 유용하고 많은 디버깅 정보를 포함한다. 하지만 이 정보는 악의적인 사용자에게도 매우 유용한 정보여서 중요 정보나 지적 재산의 유출이 발생할 위험이 있다.

예외를 처리할 때 오류가 너무 중요하다면 전파해도 된다. 하지만 예외를 전파할 경우 중요한 정보를 공개하지 않도록 주의해야 한다. 사용자에게 문제를 알리려면 무엇이 잘못되었다거나 페이지를 찾을 수 없다는 등의 일반적인 메시지를 사용해 사용자에게 문제를 알려야 한다.

##### c. 비어있는 except 블록 지양

- 비어있는 except 블록은 파이썬의 안티패턴 중에서도 가장 최악의 패턴이다.

예외를 처리할 경우 다음의 두 가지 대안을 지키는 것이 좋다.

- 보다 구체적인 예외 사용(Exception과 같이 광법위한 예외를 사용하기 보다 KeyError와 같이 구체적인 예외를 사용하자)
- except 블록에서 실제 오류 처리를 한다.

##### d. 원본 예외 포함

오류 처리 과정에서 다른 오류를 발생시키고 메시지를 변경할 수도 있다. 이 경우 원래 예외를 포함하는 것이 좋다.

예를 들어 기본 예외를 사용자 정의 예외로 래핑하고 싶다면 루트 예외에 대한 정보를 다음과 같이 포함할 수 있다.
###### 파이썬에서 예외 타입 변경시엔 raise a from b 구문을 사용한다

In [103]:
class InternalDataError(Exception):
    '''업무도메인 데이터의 예외'''
def process(data_dictionary, record_id):
    try:
        return data_dictionary[record_id]
    except KeyError as e:
        raise InternalDataError("Record not present") from e
        


파이썬 예외처리 http://hleecaster.com/python-exception/

### 파이썬에서 어썰션 사용하기
- 어썰션은 절대로 일어나지 않아하는 상황에 사용. 즉 불가능한 조건을 의미.
- 에러핸들링과 달리 프로그램을 계속할 가능성이 존재.보통 프로그램을 그냥 종료하는 것이 좋다.
- 어설션을 비즈니스 로직이랑 섞거나, 소프트웨어의 제어 흐름에 넣으면 안됨.
- 이처럼 실수를 가정해 값을 보증하는 방식으로 코딩 하기 때문에 이를 '방어적 프로그래밍'이라 부른다.

In [107]:
#assert는 뒤의 조건이 True가 아니면 AssertError를 발생한다.
a = 3
assert a == 2

AssertionError: 

In [ ]:
# Bad Case : assert 에러가 떴음에도 계속해서 프로그램을 지속하려고 들기 때문에.
try:
	assert condition.holds(), "조건에 맞지 않음"
except AssertionError:
	alternative_rocedure() 
# better: 에러 발생 후 프로그램이 종료될 것이기 때문에.
result = condition.holds()
assert result > 0, f'error: {result}

### 관심사의 분리
책임이 다르면 컴포넌트, 계층 또는 모듈로 분리되어야 한다. 프로그램의 각 부분은 자신의 관심사에 대해서만 책임을 지며 나머지 부분에 대해서는 알 필요가 없다.

소프트웨어에서 관심사를 분리하는 목표는 파급 효과를 최소화하여 유지보수성을 향상시키는 것이다. 파급 효과는 어느 지점의 변화가 프로그램 전체로 전파되는 것을 의미한다. 소프트웨어는 쉽게 변경될 수 있어야 한다.

- 응집력

응집력이란 객체가 잘 정의된 목적을 가져야 하며 가능하면 작아야 한다는 것을 의미한다. 응집력이 높을수록 코드의 재사용성은 높아진다.

- 결합력

결합력이란 두 개 이상의 객체가 서로 어떻게 의존하는지를 나타낸다. 두 개 이상의 객체간의 의존도가 높으면 낮은 재사용성과 파급효과가 발생하여 우리가 원하는 소프트웨어 디자인과는 다른 방향으로 개발될 가능성이 크다.